In [4]:
from telegram.ext import ConversationHandler
from telegram.ext import Filters
from telegram.ext import CommandHandler, MessageHandler
from telegram.ext import Updater
import telegram
from telegram import (ReplyKeyboardMarkup, ReplyKeyboardRemove)
updater = Updater(token='token', use_context=True)
dispatcher = updater.dispatcher
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)

from PIL import Image, ImageDraw, ImageFont
import textwrap

In [5]:
from escpos.printer import Usb


In [6]:

drakememe = ["Drake-Comparison.jpg", {'Drake Name': (0,300,12), 'Drake No':(600,0,12),'Drake Yes':(600,600,12)}, 100]
expandingbrain = ["Expanding-Brain.jpg", {"Small Brain": (0,0,22), "Medium Brain": (0,300,22), "Big Brain": (0,600,22), "Biggest Brain":(0,900,22)}, 40]
memes = {"Drake":drakememe, "Expanding Brain": expandingbrain}
#memes = {"Expanding Brain": expandingbrain}
#memes{'drake'} #.append({'Drake No':[200,200],'Drake Yes':[200,400]})
memes



{'Drake': ['Drake-Comparison.jpg',
  {'Drake Name': (0, 300, 12),
   'Drake No': (600, 0, 12),
   'Drake Yes': (600, 600, 12)},
  100],
 'Expanding Brain': ['Expanding-Brain.jpg',
  {'Small Brain': (0, 0, 22),
   'Medium Brain': (0, 300, 22),
   'Big Brain': (0, 600, 22),
   'Biggest Brain': (0, 900, 22)},
  40]}

In [7]:
str(len(memes[str("Expanding Brain")][1]))
currentmeme = "Drake"



In [8]:




  
CONTENT, INFGAT, FIN = range(3)

def start(update, context):
    user_data = {}
    #context.bot.send_message(chat_id=update.effective_chat.id, text=message)
    message = "Is this message " + listcategories(" or ", memes) + " ?"

    reply_markup = telegram.ReplyKeyboardMarkup([list(memes.keys())])

    context.bot.send_message(chat_id=update.effective_chat.id, 
             text=message, 
             reply_markup=reply_markup, one_time_keyboard=True)

    
    print("start done")
    return CONTENT
    
def content(update, context):
    print("content")
    message = "Ok. This message has " + str(len(memes[str(update.message.text)][1])) + " different text fields: \n"
    for i in memes[str(update.message.text)][1]:
        message += i + " \n"
    
    
    context.bot.send_message(chat_id=update.effective_chat.id, 
             text=message, 
             reply_markup= telegram.ReplyKeyboardRemove())
    
    context.user_data.update({'Meme': update.message.text})
    context.user_data.update({'Textfield': 0})
    
    currentmeme = context.user_data["Meme"]
    textfieldnumber = context.user_data["Textfield"]
    message = "We will start with " + list(memes[currentmeme][1].keys())[textfieldnumber]
    message += ". Please enter the text for this field now."
    context.bot.send_message(chat_id=update.effective_chat.id, 
             text=message)
    
    

    return INFGAT

def textgatherer(update, context) :
    currentmeme = context.user_data["Meme"]
    textfieldnumber = context.user_data["Textfield"]
    fieldname = list(memes[currentmeme][1].keys())[textfieldnumber]
    
    context.user_data.update({fieldname : update.message.text})
    
    context.user_data.update({'Textfield': context.user_data["Textfield"] + 1})
    textfieldnumber = context.user_data["Textfield"]
    
    print("The Current Meme: " + currentmeme)
    print("Textfieldnumber " + str(textfieldnumber))
    print("fieldname " + fieldname)
    print(str(context.user_data))
    
    if context.user_data["Textfield"] < len(memes[currentmeme][1]):
        message = "Ok. Now: " + list(memes[currentmeme][1].keys())[textfieldnumber]
        message += "Please enter the text for this field now."

        context.bot.send_message(chat_id=update.effective_chat.id, 
                 text=message)
        
    else:
        context.bot.send_message(chat_id=update.effective_chat.id, 
                 text="your coder boy fucked up. this message should never appear")
    
    if context.user_data["Textfield"] < len(memes[currentmeme][1]) - 1:
        return INFGAT
    else:
        return FIN
    
    
    
    

#def mtype(update, context):
#    #message = "You will print: " + context.user_data['text'] 
#    message = update.message.text + " it is ! What Priority should be assigned ?"
#    context.user_data.update({'cat': update.message.text})
#    reply_markup = telegram.ReplyKeyboardMarkup(priorities)
#    context.bot.send_message(chat_id=update.effective_chat.id, text=message, reply_markup = reply_markup)
    
#    #return ConversationHandler.END
#    return FIN

def fin(update, context):
    print("FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN")
    
    
    currentmeme = context.user_data["Meme"]
    textfieldnumber = context.user_data["Textfield"]
    fieldname = list(memes[currentmeme][1].keys())[textfieldnumber]
    
    context.user_data.update({fieldname : update.message.text})
    
    context.user_data.update({'Textfield': context.user_data["Textfield"] + 1})
    textfieldnumber = context.user_data["Textfield"]
    
    print("The Current Meme: " + currentmeme)
    print("Textfieldnumber " + str(textfieldnumber))
    print("fieldname " + fieldname)
    print(str(context.user_data))
    
    #context.user_data.update({'pri': update.message.text})
    message = "Ok. Your message is done. You entered the following parameters: \n"
    for i in context.user_data:
        message += (i + " : " +   str(context.user_data[i])) + "\n"
        
    context.bot.send_message(chat_id=update.effective_chat.id, text=message, reply_markup = telegram.ReplyKeyboardRemove())
    print(type(context.user_data))
    
    makememe(context.user_data)
    context.bot.send_photo(chat_id=update.effective_chat.id, photo=open('test.png', 'rb'))
    
    return ConversationHandler.END
    
    

conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],

        states={
            CONTENT: [MessageHandler(Filters.text, content)],
            INFGAT: [MessageHandler(Filters.text, textgatherer)],
            FIN: [MessageHandler(Filters.text, fin)]
        },

        fallbacks=[CommandHandler('cancel', start)]
    )

    
        
dispatcher.add_handler(conv_handler)
        

    
    
        
    


In [9]:
def listcategories(divider, divlist):
    s = ""
    for i in divlist:
        s += i
        s += divider
            
    s = s[:-len(divider)]
    return s




In [10]:
from telegram.ext import CommandHandler
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)


def caps(update, context):
    text_caps = ' '.join(context.args).upper()
    context.bot.send_message(chat_id=update.effective_chat.id, text=text_caps)


caps_handler = CommandHandler('caps', caps)
dispatcher.add_handler(caps_handler)



#from telegram.ext import MessageHandler, Filters
#echo_handler = MessageHandler(Filters.text, echo)
#dispatcher.add_handler(echo_handler)


In [11]:
def makememe(user_data):
    memeinfo = memes[user_data["Meme"]]
    im = Image.open(memeinfo[0])
    draw = ImageDraw.Draw(im)
    
    smallfont = ImageFont.truetype("sfont.otf", memeinfo[2])
    for i in memeinfo[1]:
        #memeinfo[1][i] cords
        print(i)
        wrap(user_data[i], memeinfo[1][i][0], memeinfo[1][i][1], draw, smallfont, memeinfo[1][i][2])

    im.save("test.png")
    
    


def rmaxwidth(text, width, font):
    print("Text: " + text + " Width: " + str(width))
    for i in range(0, len(text)):
        if font.getsize(text[:i])[0] > width:
            print("returning: " + text[:i-1])
            print("because " + str(font.getsize(text[:i])[0]))
            
            return text[:i-1]
    print("returning whole text")
    return text


def wrap(text, x, y, draw, font, margin):
    for line in textwrap.wrap(text, width=margin):
        draw.text((x, y), line, font=font, fill="#000000")
        y += font.getsize(line)[1]

In [12]:
updater.start_polling()

2020-02-26 22:23:17,261 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/dispatcher.py", line 390, in process_update
    handler.handle_update(update, self, check, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/conversationhandler.py", line 381, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/handler.py", line 117, in handle_update
    return self.callback(update, context)
  File "<ipython-input-8-39fe91b357a7>", line 20, in content
    message = "Ok. This message has " + str(len(memes[str(update.message.text)][1])) + " different text fields: \n"
KeyError: '1'
2020-02-26 22:23:17,267 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent cal

start done
content
content
start done
content
content
content
content
The Current Meme: Drake
Textfieldnumber 1
fieldname Drake Name
{'Meme': 'Drake', 'Textfield': 1, 'Drake Name': '1'}
The Current Meme: Drake
Textfieldnumber 2
fieldname Drake No
{'Meme': 'Drake', 'Textfield': 2, 'Drake Name': '1', 'Drake No': '2'}
FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN
The Current Meme: Drake
Textfieldnumber 3
fieldname Drake Yes
{'Meme': 'Drake', 'Textfield': 3, 'Drake Name': '1', 'Drake No': '2', 'Drake Yes': '3'}
<class 'dict'>
Drake Name
Drake No
Drake Yes
start done
content
The Current Meme: Drake
Textfieldnumber 1
fieldname Drake Name
{'Meme': 'Drake', 'Textfield': 1, 'Drake Name': 'df hrtoueh t öioert a jraeriot kj', 'Drake No': '2', 'Drake Yes': '3'}
The Current Meme: Drake
Textfieldnumber 2
fieldname Drake No
{'Meme': 'Drake', 'Textfield': 2, 'Drake Name': 'df hrtoueh t öioert a jraeriot kj', 'Drake No': 'oth wrrrlie th erit  wjht wret', 'Drake Yes': '3'}
FIN---FIN---FIN---FIN---

2020-02-26 22:24:25,848 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/dispatcher.py", line 390, in process_update
    handler.handle_update(update, self, check, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/conversationhandler.py", line 381, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/handler.py", line 117, in handle_update
    return self.callback(update, context)
  File "<ipython-input-8-39fe91b357a7>", line 20, in content
    message = "Ok. This message has " + str(len(memes[str(update.message.text)][1])) + " different text fields: \n"
KeyError: 'Brettspiel Chat ?'


content


2020-02-26 22:24:30,400 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/dispatcher.py", line 390, in process_update
    handler.handle_update(update, self, check, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/conversationhandler.py", line 381, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "/home/ppasch/.local/lib/python3.7/site-packages/telegram/ext/handler.py", line 117, in handle_update
    return self.callback(update, context)
  File "<ipython-input-8-39fe91b357a7>", line 20, in content
    message = "Ok. This message has " + str(len(memes[str(update.message.text)][1])) + " different text fields: \n"
KeyError: 'Meme chat'


content
content
The Current Meme: Drake
Textfieldnumber 1
fieldname Drake Name
{'Meme': 'Drake', 'Textfield': 1, 'Drake Name': 'Chat'}
The Current Meme: Drake
Textfieldnumber 2
fieldname Drake No
{'Meme': 'Drake', 'Textfield': 2, 'Drake Name': 'Chat', 'Drake No': 'Brettspiel chat'}
FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN---FIN
The Current Meme: Drake
Textfieldnumber 3
fieldname Drake Yes
{'Meme': 'Drake', 'Textfield': 3, 'Drake Name': 'Chat', 'Drake No': 'Brettspiel chat', 'Drake Yes': 'Meme chat'}
<class 'dict'>
Drake Name
Drake No
Drake Yes
start done
content
The Current Meme: Expanding Brain
Textfieldnumber 1
fieldname Small Brain
{'Meme': 'Expanding Brain', 'Textfield': 1, 'Drake Name': 'df hrtoueh t öioert a jraeriot kj', 'Drake No': 'oth wrrrlie th erit  wjht wret', 'Drake Yes': 'w tuoehaWTR  WERUTH WERT KJHSDG HJ', 'Small Brain': 'Meme Maker benutzen'}
The Current Meme: Expanding Brain
Textfieldnumber 2
fieldname Medium Brain
{'Meme': 'Expanding Brain', 'Textfield': 2, 'Dr

In [11]:
updater.stop()

In [11]:

user_data = {'Meme': 'Expanding Brain', 'Textfield': 4, 'Drake No': 'c1', 'Drake Yes': 'c2', 'pri': 'c2', 'Small Brain': 'small brain text placeholder important for testing', 'Medium Brain': 'medium brain text placeholder important for testing', 'Big Brain': 'big brain text placeholder important for testing', 'Biggest Brain': 'biggest brain text placeholder important for testing'}

In [12]:
user_data

{'Meme': 'Expanding Brain',
 'Textfield': 4,
 'Drake No': 'c1',
 'Drake Yes': 'c2',
 'pri': 'c2',
 'Small Brain': 'small brain text placeholder important for testing',
 'Medium Brain': 'medium brain text placeholder important for testing',
 'Big Brain': 'big brain text placeholder important for testing',
 'Biggest Brain': 'biggest brain text placeholder important for testing'}

In [13]:
memes

{'Expanding Brain': ['Expanding-Brain.jpg',
  {'Small Brain': (0, 0, 22),
   'Medium Brain': (0, 300, 22),
   'Big Brain': (0, 600, 22),
   'Biggest Brain': (0, 900, 22)},
  40]}

In [14]:
memeinfo = memes[user_data["Meme"]]
memeinfo

['Expanding-Brain.jpg',
 {'Small Brain': (0, 0, 22),
  'Medium Brain': (0, 300, 22),
  'Big Brain': (0, 600, 22),
  'Biggest Brain': (0, 900, 22)},
 40]

In [15]:
im = Image.open(memeinfo[0])
draw = ImageDraw.Draw(im)

In [16]:
smallfont = ImageFont.truetype("sfont.otf", memeinfo[2])
for i in memeinfo[1]:
    #memeinfo[1][i] cords
    print(i)
    wrap(user_data[i], memeinfo[1][i][0], memeinfo[1][i][1], draw, smallfont, memeinfo[1][i][2])

im.save("test.png")

Small Brain
Medium Brain
Big Brain
Biggest Brain


In [17]:
def makememe(user_data):
    memeinfo = memes[user_data["Meme"]]
    im = Image.open(memeinfo[0])
    draw = ImageDraw.Draw(im)
    
    smallfont = ImageFont.truetype("sfont.otf", memeinfo[2])
    for i in memeinfo[1]:
        #memeinfo[1][i] cords
        print(i)
        wrap(user_data[i], memeinfo[1][i][0], memeinfo[1][i][1], draw, smallfont, memeinfo[1][i][2])

    im.save("test.png")
    
    


def rmaxwidth(text, width, font):
    print("Text: " + text + " Width: " + str(width))
    for i in range(0, len(text)):
        if font.getsize(text[:i])[0] > width:
            print("returning: " + text[:i-1])
            print("because " + str(font.getsize(text[:i])[0]))
            
            return text[:i-1]
    print("returning whole text")
    return text


def wrap(text, x, y, draw, font, margin):
    for line in textwrap.wrap(text, width=margin):
        draw.text((x, y), line, font=font, fill="#000000")
        y += font.getsize(line)[1]

In [17]:
memes = {"drake":1, "tupac":2, "boyfried":3, "brain":4, "bob" : 5}
r = []
if len(memes) % 2 != 1:
    for i in range(0, len(memes), 2):
        r.append([list(memes.keys())[i], list(memes.keys())[i + 1] ])
else:
    for i in range(0, len(memes) - 1, 2):
        r.append([list(memes.keys())[i], list(memes.keys())[i + 1] ])
    r.append([list(memes.keys())[-1]])
return r

[['drake', 'tupac'], ['boyfried', 'brain'], ['bob']]